## Load Results

In [2]:
run_code = "07"

# JUPYTER
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import pickle
import utils.plot_utils as plot
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from itertools import cycle
results_dir = os.getcwd() + "/results/" + run_code
outputpath = results_dir + "/output.pickle"
with open(outputpath, "rb") as f:
    df = pickle.load(f)
with open(results_dir + "/opts.pickle", "rb") as f:
    opts = pickle.load(f)
with open(opts["picklepath"], "rb") as f:
    data = pickle.load(f)
if os.path.isfile(results_dir + "/output_julia.json"):
    FLAG_JULIA = True
    with open(results_dir + "/output_julia.json") as f:
        julia = json.load(f)
    julia = pd.DataFrame(julia)
    df = df.join(julia)
else:
    FLAG_JULIA = False
df["t_centre"] = (df["t_start"]+df["t_end"])/2

## Plots

In [ ]:
###   DIMENSIONALITY VS TIME   ###
##################################
fig, ax = plt.subplots(layout="constrained")
fig.set_size_inches(6, 4)
ax.plot(df["t_centre"], df["m"], label="Cao embedding dimension")
ax.plot(df["t_centre"], df["KYD"], label="Kaplan-Yorke Dimension")
if FLAG_JULIA:
    ax.plot(df["t_centre"], df["m_kraemer"], label="Kraemer (2021) embedding dimension")
fig.suptitle("Dimension vs time")
fig.legend(loc="lower left", fontsize="8")
ax.set_ylabel("Dimension")
ax.set_xlabel("Time")
# ax.set_ylim(5, 8)
plot.watermark(ax, data, color="white")
if "ROIs" in opts["exp_meta"]:
    plot.roi_spans(ax, opts["exp_meta"]["ROIs"])
    # clr_cycler = cycle([(.5,.5,.5), (.65, .65, .65)])
    # for i, ROI in enumerate(opts["exp_meta"]["ROIs"]):
    #     wind = windf.iloc[i]
    #     annot_string = "$m_{cao}="+str(int(wind["m"]))+"$\n$m_{kra}="+str(int(wind["m_kraemer"]))+"$\n$KYD="+str(round(wind["KYD"],2))+"$"
    #     ax.annotate(annot_string, ((ROI[1]+ROI[0])/2, .86), xycoords=("data", "figure fraction"), ha="center", va="top", fontsize="8", color=(.5,.5,.5))

In [ ]:
###   CAO SUMMARY FOR WINDOW   ###
##################################
from utils.load_utils import normalise_shear
row = df.iloc[20]
data_ = normalise_shear(data[row["win_start"]:row["win_end"]].copy())
plot.summary_calc_m_tau(data_, opts["dynsys_params"]["tau_to_try"], opts["dynsys_params"]["E1_thresh"], row["E1s"], row["E2s"], row["eps_over_L"], int(row["m"]), int(row["tau"]))

In [ ]:
###   SHANNON ENTROPY   ###
############################
fig, ax = plt.subplots(layout="constrained")
ax.plot(df["t_centre"], df["shannon_entropy_vb"])
fig.suptitle("Entropy vs time")
ax.set_ylabel("Shannon entropy")
ax.set_xlabel("Time")
plot.watermark(ax, data)
if "ROIs" in opts["exp_meta"]:
    plot.roi_spans(ax, opts["exp_meta"]["ROIs"])